In [2]:
from dotenv import load_dotenv
load_dotenv()

import streamlit as st
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get repsonses
model=genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])
def get_gemini_response(question):

    response=chat.send_message(question,stream=True)
    return response

##initialize our streamlit app

st.set_page_config(page_title="Q&A Demo")

st.header("Gemini LLM Application")

# Initialize session state for chat history if it doesn't exist
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

input=st.text_input("Input: ",key="input")
submit=st.button("Ask the question")

if submit and input:
    response=get_gemini_response(input)
    # Add user query and response to session state chat history
    st.session_state['chat_history'].append(("You", input))
    st.subheader("The Response is")
    for chunk in response:
        st.write(chunk.text)
        st.session_state['chat_history'].append(("Bot", chunk.text))
st.subheader("The Chat History is")

for role, text in st.session_state['chat_history']:
    st.write(f"{role}: {text}")





2024-06-21 03:29:06.144 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-21 03:29:06.162 Session state does not function when running a script without `streamlit run`


In [4]:
!pip install streamlit google-generativeai pyngrok


In [5]:
%%writefile app.py
import streamlit as st
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get repsonses
model = genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])
def get_gemini_response(question):
    response = chat.send_message(question, stream=True)
    return response

##initialize our streamlit app
st.set_page_config(page_title="Q&A Demo")

st.header("Gemini LLM Application")

# Initialize session state for chat history if it doesn't exist
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

input = st.text_input("Input: ", key="input")
submit = st.button("Ask the question")

if submit and input:
    response = get_gemini_response(input)
    # Add user query and response to session state chat history
    st.session_state['chat_history'].append(("You", input))
    st.subheader("The Response is")
    for chunk in response:
        st.write(chunk.text)
        st.session_state['chat_history'].append(("Bot", chunk.text))

st.subheader("The Chat History is")
for role, text in st.session_state['chat_history']:
    st.write(f"{role}: {text}")


Writing app.py


In [10]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get responses
model = genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])

def get_gemini_response(question):
    response = chat.send_message(question, stream=True)
    full_response = "".join([chunk.text for chunk in response])
    return full_response

# Gradio interface
def gradio_interface(input_text):
    response = get_gemini_response(input_text)
    return response

# Create a Gradio interface
interface = gr.Interface(fn=gradio_interface,
                         inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your question here..."),
                         outputs="text",
                         title="Gemini LLM Application",
                         description="Ask questions and get responses from the Gemini LLM.")

# Launch the Gradio interface
interface.launch()



AttributeError: module 'gradio' has no attribute 'inputs'

In [11]:
!pip install google-generativeai translate


In [17]:
import os
import google.generativeai as genai
import re
from translate import Translator

# Set the Gemini API key (replace with your actual key)
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def extract_locations(user_input_text):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input_text)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the Gemini API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = genai.generate_text(
        prompt,
        model="gemini-pro",
        max_output_tokens=500
    )

    return response.text.strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    # Use a different variable name to avoid overwriting the input() function
    user_input_text = input("Enter your request: ")
    if user_input_text.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input_text)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.


TypeError: 'str' object is not callable

In [18]:
import os
import google.generativeai as genai
from translate import Translator

# Set the Gemini API key (replace with your actual key)
os.environ["GOOGLE_API_KEY"] = ""
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [19]:
def generate_route_description(start_location, end_location):
    # Prompt for the Gemini API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = genai.generate_text(
        prompt,
        model="gemini-pro",
        max_output_tokens=500
    )

    return response.text.strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = [translator.translate(sentence) for sentence in sentences]

    translated_description = '. '.join(translated_sentences)

    return translated_description


In [20]:
import re

def extract_locations(user_input_text):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input_text)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input_text = input("Enter your request: ")
    if user_input_text.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input_text)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.


TypeError: 'str' object is not callable

In [21]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 19.7 MB/s eta 0:00:00


In [22]:
import google.generativeai as genai
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)


In [27]:
!pip install openai google-generativeai translate pydub SpeechRecognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 20.8 MB/s eta 0:00:00


In [4]:
import os
import openai
import re
from translate import Translator
from pydub import AudioSegment
import speech_recognition as sr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_path)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_whisper(audio_data, model="base")
    return text

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    # Assuming audio file input for simplicity. You can change this to live audio recording if needed.
    audio_path = input("Enter the path to the audio file (or type 'exit' to quit): ")
    if audio_path.lower() == 'exit':
        print('Exiting')
        break

    user_input = transcribe_audio(audio_path)
    print(f"Transcribed Text: {user_input}")

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter the path to the audio file (or type 'exit' to quit): /content/record1.in-speech.mp3
Transcribed Text:  Provide route from Mumbai to Puna.
Route Description (in English): Sure! The driving route from Mumbai to Pune is approximately 150 kilometers and can take around 3 to 4 hours, depending on traffic conditions. Here's a detailed route with major turns, landmarks, and distances:

1. Start in Mumbai and head towards the Eastern Express Highway.

2. Follow the Eastern Express Highway for about 25 kilometers until you reach the town of Thane.

3. After crossing Thane, continue on the Eastern Express Highway towards the Majiwada Junction.

4. At Majiwada Junction, take a left turn onto the Mumbai-Nashik Expressway and continue for about 10 kilometers.

5. Keep following the Mumbai-Nashik Expressway until you reach the Khalapur Toll Plaza at around 65 kilometers from Mumbai.

6. After crossing the toll pla

In [5]:
import os
import openai
import re
from translate import Translator
from pydub import AudioSegment
import speech_recognition as sr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_path)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_whisper(audio_data, model="base")
    return text

# Run the chatbot
print("Greetings! I am SARATHI, developed by NIT Calicut students.")
while True:
    input_type = input("Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): ").lower()

    if input_type == 'exit':
        print('Exiting')
        break

    elif input_type == 'text':
        user_input = input("Enter your request: ")

        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
            continue

        route_description = generate_route_description(start_location, end_location)
        print("Route Description (in English):", route_description)

        target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
        translated_description = translate_description(route_description, target_language)
        print(f"Route Description (in {target_language}):", translated_description)

    elif input_type == 'audio':
        audio_path = input("Enter the path to the audio file: ")

        try:
            user_input = transcribe_audio(audio_path)
            print(f"Transcribed Text: {user_input}")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Invalid input format from audio transcription.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description (in English):", route_description)

            target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
            translated_description = translate_description(route_description, target_language)
            print(f"Route Description (in {target_language}):", translated_description)

        except Exception as e:
            print(f"Error processing audio: {str(e)}")
            continue

    else:
        print("Invalid input. Please enter 'text' or 'audio'.")


Greetings! I am SARATHI, developed by NIT Calicut students.
Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): text
Enter your request: provide route from kannur to kozhikode
Route Description (in English): Certainly! The driving route from Kannur to Kozhikode is about 95 kilometers long and takes approximately 2 to 3 hours, depending on traffic conditions. Here is a detailed route with major turns, landmarks, and approximate distances:

1. Start from Kannur and head south on NH 66 (also known as Kochi-Panvel Highway).
2. After about 10 kilometers, you will pass through Thalassery. Continue on NH 66.
3. About 30 kilometers from Kannur, you will reach Mahe. Stay on NH 66.
4. After driving about 30 more kilometers, you will approach Vadakara. Continue on NH 66.
5. Around 10 kilometers after Vadakara, you will reach the Feroke Bridge on the outskirts of Kozhikode city.
6. Cross the Feroke Bridge and continue straight on NH 66.
7. Drive for another 10 kilometers,

In [6]:
import os
import openai
import re
from translate import Translator
from pydub import AudioSegment
import speech_recognition as sr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each chunk of sentences and join them back together
    for chunk in chunks(sentences, 5):  # Limiting to 5 sentences per chunk to avoid length issues
        chunk_text = '. '.join(chunk)
        translated_chunk = translator.translate(chunk_text)
        translated_sentences.append(translated_chunk)

    translated_description = '. '.join(translated_sentences)

    return translated_description

def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_path)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_whisper(audio_data, model="base")
    return text

# Run the chatbot
print("Greetings! I am SARATHI, developed by NIT Calicut students.")
while True:
    input_type = input("Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): ").lower()

    if input_type == 'exit':
        print('Exiting')
        break

    elif input_type == 'text':
        user_input = input("Enter your request: ")

        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
            continue

        route_description = generate_route_description(start_location, end_location)
        print("Route Description (in English):", route_description)

        if len(route_description) > 500:
            print("Route description exceeds 500 characters. Please provide a shorter request.")
            continue

        target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
        translated_description = translate_description(route_description, target_language)
        print(f"Route Description (in {target_language}):", translated_description)

    elif input_type == 'audio':
        audio_path = input("Enter the path to the audio file: ")

        try:
            user_input = transcribe_audio(audio_path)
            print(f"Transcribed Text: {user_input}")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Invalid input format from audio transcription.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description (in English):", route_description)

            if len(route_description) > 500:
                print("Route description exceeds 500 characters. Please provide a shorter request.")
                continue

            target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
            translated_description = translate_description(route_description, target_language)
            print(f"Route Description (in {target_language}):", translated_description)

        except Exception as e:
            print(f"Error processing audio: {str(e)}")
            continue

    else:
        print("Invalid input. Please enter 'text' or 'audio'.")


Greetings! I am SARATHI, developed by NIT Calicut students.
Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): audio
Enter the path to the audio file: /content/record1.in-speech.mp3
Transcribed Text:  Provide route from Mumbai to Puna.
Route Description (in English): Sure! The route from Mumbai to Pune is approximately 150 kilometers and can take around 3 to 4 hours, depending on traffic conditions. Here is a detailed driving route from Mumbai to Pune:

1. **Starting Point - Mumbai**:
   - Start from Mumbai and head towards the Mumbai-Pune Expressway. You can take the Eastern Express Highway or Western Express Highway to reach the expressway.

2. **Kharghar Toll Plaza**:
   - After around 35-40 kilometers on the expressway, you will come across the Kharghar Toll Plaza. Pay the toll and continue on the Mumbai-Pune Expressway.

3. **Lonavala/Khandala Exit (optional)**:
   - After driving for about 70-80 kilometers, you will see the exit for Lonavala/Khandala. I

In [ ]:
udio


In [7]:
import os
import openai
import re
from translate import Translator
from pydub import AudioSegment
import speech_recognition as sr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into chunks of approximately 500 characters
    def chunks(lst, max_chunk_length):
        chunk = []
        current_length = 0
        for sentence in lst:
            sentence_length = len(sentence)
            if current_length + sentence_length > max_chunk_length:
                yield ' '.join(chunk)
                chunk = [sentence]
                current_length = sentence_length
            else:
                chunk.append(sentence)
                current_length += sentence_length + 1  # Adding 1 for the space between sentences
        if chunk:
            yield ' '.join(chunk)

    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', route_description)
    translated_sentences = []

    # Translate each chunk of sentences and join them back together
    for chunk in chunks(sentences, 500):
        translated_chunk = translator.translate(chunk)
        translated_sentences.append(translated_chunk)

    translated_description = ' '.join(translated_sentences)

    return translated_description

def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_path)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_whisper(audio_data, model="base")
    return text

# Run the chatbot
print("Greetings! I am SARATHI, developed by NIT Calicut students.")
while True:
    input_type = input("Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): ").lower()

    if input_type == 'exit':
        print('Exiting')
        break

    elif input_type == 'text':
        user_input = input("Enter your request: ")

        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
            continue

        route_description = generate_route_description(start_location, end_location)
        print("Route Description (in English):", route_description)

        target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
        translated_description = translate_description(route_description, target_language)
        print(f"Route Description (in {target_language}):", translated_description)

    elif input_type == 'audio':
        audio_path = input("Enter the path to the audio file: ")

        try:
            user_input = transcribe_audio(audio_path)
            print(f"Transcribed Text: {user_input}")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Invalid input format from audio transcription.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description (in English):", route_description)

            target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
            translated_description = translate_description(route_description, target_language)
            print(f"Route Description (in {target_language}):", translated_description)

        except Exception as e:
            print(f"Error processing audio: {str(e)}")
            continue

    else:
        print("Invalid input. Please enter 'text' or 'audio'.")


Greetings! I am SARATHI, developed by NIT Calicut students.
Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): text
Enter your request: from mumbai to pune
Route Description (in English): Sure! Here is a detailed driving route from Mumbai to Pune:

1. Start in Mumbai and get on the Mumbai-Pune Expressway from the nearest entry point.
2. Drive on the Mumbai-Pune Expressway for approximately 93 miles (150 km).
3. Keep driving on the Expressway until you reach the Lonavala exit. Take the Lonavala exit to merge onto Old Mumbai Pune Highway/State Highway 54.
4. Continue on the Old Mumbai Pune Highway for 6.5 miles (10.5 km) until you reach Lonavala town.
5. In Lonavala, drive through the town on the Old Mumbai Pune Highway.
6. As you exit Lonavala, continue on the Old Mumbai Pune Highway for another 26 miles (42 km) towards Pune.
7. Pass through the towns of Kamshet and Talegaon Dabhade on the way to Pune.
8. As you approach Pune, you will pass by Pune University 

In [8]:
!pip install pyaudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


In [9]:
import os
import openai
import re
from translate import Translator
from pydub import AudioSegment
import speech_recognition as sr
import pyaudio
import wave

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into chunks of approximately 500 characters
    def chunks(lst, max_chunk_length):
        chunk = []
        current_length = 0
        for sentence in lst:
            sentence_length = len(sentence)
            if current_length + sentence_length > max_chunk_length:
                yield ' '.join(chunk)
                chunk = [sentence]
                current_length = sentence_length
            else:
                chunk.append(sentence)
                current_length += sentence_length + 1  # Adding 1 for the space between sentences
        if chunk:
            yield ' '.join(chunk)

    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', route_description)
    translated_sentences = []

    # Translate each chunk of sentences and join them back together
    for chunk in chunks(sentences, 500):
        translated_chunk = translator.translate(chunk)
        translated_sentences.append(translated_chunk)

    translated_description = ' '.join(translated_sentences)

    return translated_description

def transcribe_audio():
    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Define parameters for audio recording
    format = pyaudio.paInt16
    channels = 1
    rate = 16000  # Sample rate
    chunk = 1024  # Size of each audio chunk

    # Create an input stream
    stream = audio.open(format=format,
                        channels=channels,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

    print("Recording audio...")

    frames = []

    # Record audio in chunks and save to frames list
    while True:
        try:
            data = stream.read(chunk)
            frames.append(data)
        except KeyboardInterrupt:
            print("Recording stopped.")
            break

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded audio to a WAV file
    wave_output_file = "recorded_audio.wav"
    with wave.open(wave_output_file, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

    return wave_output_file

# Run the chatbot
print("Greetings! I am SARATHI, developed by NIT Calicut students.")
while True:
    input_type = input("Enter 'text' for text input or 'audio' for audio input (type 'exit' to quit): ").lower()

    if input_type == 'exit':
        print('Exiting')
        break

    elif input_type == 'text':
        user_input = input("Enter your request: ")

        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
            continue

        route_description = generate_route_description(start_location, end_location)
        print("Route Description (in English):", route_description)

        target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
        translated_description = translate_description(route_description, target_language)
        print(f"Route Description (in {target_language}):", translated_description)

    elif input_type == 'audio':
        try:
            audio_file = transcribe_audio()
            print(f"Audio recording saved to: {audio_file}")

            # Perform transcription and further processing with the recorded audio
            user_input = transcribe_audio(audio_file)
            print(f"Transcribed Text: {user_input}")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Invalid input format from audio transcription.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description (in English):", route_description)

            target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()
            translated_description = translate_description(route_description, target_language)
            print(f"Route Description (in {target_language}):", translated_description)

        except Exception as e:
            print(f"Error processing audio: {str(e)}")
            continue

    else:
        print("Invalid input. Please enter 'text' or 'audio'.")


ModuleNotFoundError: No module named 'pyaudio'

In [1]:
!pip install openai==0.28

In [2]:
!pip install openai-whisper

In [1]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):",translated_description)

Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install openai
!pip install google

In [23]:
model = genai.GenerativeModel('gemini-pro')

In [25]:
question = 'What is the capital of Argentina?'
response = model.generate_content(question)
response.resolve()
response.text
chat = model.start_chat(history=[])
countries = ['Chile', 'Bolivia']

for country in countries:
    message = f'What is the capital of {country}?'
    chat.send_message(message)

chat.history
response = chat.send_message('From which countries did I ask you the capital city?')
response.resolve()
response.text

2024-06-21 03:49:27.798 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4410.82ms
2024-06-21 03:49:29.672 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1871.06ms
2024-06-21 03:49:32.714 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3035.74ms
2024-06-21 03:49:35.582 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2859.09ms


'Chile and Bolivia'

In [26]:
import time
import gradio as gr

chat = model.start_chat(history=[])

# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": [{"text": chat[0]}], "role": "user"})
        new_history.append({"parts": [{"text": chat[1]}], "role": "model"})
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat.history = transform_history(history)
    response = chat.send_message(message)
    response.resolve()

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.05)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 title='Gemini Chat',
                 textbox=gr.Textbox(placeholder="Question to Gemini"),
                 retry_btn=None).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f125722cbf60a65a2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-06-21 03:50:42.588 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3221.88ms
2024-06-21 03:51:40.612 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 5618.42ms
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1526, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 656, in async_iteration
    return await iterator._

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f125722cbf60a65a2e.gradio.live


In [24]:
!pip install --upgrade gradio

In [9]:
!pip install gradio google-generativeai
!pip install --upgrade gradio

In [3]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [1]:
!pip install streamlit
!pip install google-generativeai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
